In [1]:
import pandas as pd
import re
import itertools
import numpy as np
import random
from sklearn.model_selection import train_test_split

%matplotlib inline

random.seed(0)
np.random.seed(0)

mu, sigma = 0, 0.1 

# Análise dos dados de precedentes do STJ

Levando em consideração que cada jurisprudência do conjunto de dados terá no mínimo 5 precedentes, utilizando a fórmula de combinação 𝐶(𝑛,𝑟)=𝑛!/(𝑟!(𝑛−𝑟)!), onde n=10 e r=2, temos que para cada jurisprudência podemos gerar no mínimo 10 pares de precedentes sem repetição.

In [2]:
stj = pd.read_csv('../datasets/jurisprudencias_stj_final.csv')

In [3]:
stj_sts_data = []
for group_name, tema_group in stj.groupby('TEMA'):
    pares = list(set(itertools.combinations(tema_group.EMENTA, 2)))
    noise = np.random.normal(mu, sigma, len(pares))
    scores = [4.5]*len(pares)+noise
    for i, par in enumerate(pares):
        stj_sts_data.append([par[0], par[1], scores[i], 4])

precedentes_mesma_jurisprudencia = len(stj_sts_data)
print('Total de pares de precedentes da mesma Jurisprudência gerados: ', precedentes_mesma_jurisprudencia)


Total de pares de precedentes da mesma Jurisprudência gerados:  18217


Agora preciso gerar também pares de precedentes que não são similares. Para isso posso usar a informação da Matéria de cada precedente, e gerar pares entre Matérias diferentes, o que garante que a similaridade entre os precedentes seja praticamente nula. Como os pares não similares gerados por essa abordagem será maior que a quantidade de pares da mesma jurisprudência gerados pelo passo anterior, uso a relação da quantidade de pares precedentes da mesma jurisprudência, e os pares de Matérias diferentes para manter um conjunto de dados balanceado.

In [4]:
#gero pares de matérias diferentes
pares_materias = list(set(itertools.combinations(stj.MATERIA.unique().tolist(), 2)))
print('Total de pares de diferentes Matérias: ' , len(pares_materias))
total_por_par_materias = int(precedentes_mesma_jurisprudencia/len(pares_materias))
print('Total de pares a serem usados a partir de cada par de Matéria: ', total_por_par_materias)
for materias_par in list(set(itertools.combinations(stj.MATERIA.unique().tolist(), 2))):
    #recupero as ementas das duas diferentes matérias
    precedentes_materias = [stj[stj.MATERIA == materia].EMENTA.tolist() for materia in materias_par]
    # gero um produto cartesiano entre as ementas das diferentes matérias
    pares = list(itertools.product(*precedentes_materias))[:total_por_par_materias]
    noise = np.random.normal(mu, sigma, len(pares))
    scores = [0.5]*len(pares)+noise
    for i, par in enumerate(pares):
        stj_sts_data.append([par[0], par[1], scores[i], 0])
print('Total de pares até o momento: ', len(stj_sts_data))

Total de pares de diferentes Matérias:  21
Total de pares a serem usados a partir de cada par de Matéria:  867
Total de pares até o momento:  36424


In [5]:
pares_natureza = []
for group_name, natureza_group in stj.groupby('NATUREZA'):
    for temas_pares in list(set(itertools.combinations(natureza_group.TEMA.unique(), 2))):
        precedentes_temas = [natureza_group[natureza_group.TEMA == tema].EMENTA.tolist() 
                                for tema in temas_pares]
        pares = list(itertools.product(*precedentes_temas))
        noise = np.random.normal(mu, sigma, len(pares))
        scores = [3]*len(pares)+noise
        for i, par in enumerate(pares):
            pares_natureza.append([par[0], par[1], scores[i], 3])

In [6]:
samples_pares_natureza = random.sample(pares_natureza, min(precedentes_mesma_jurisprudencia, len(pares_natureza)))
stj_sts_data = stj_sts_data + samples_pares_natureza
print('Total de pares até o momento', len(stj_sts_data))

Total de pares até o momento 54641


In [7]:
stj_sts_df = pd.DataFrame(stj_sts_data, columns=['sentence_A','sentence_B','score','range'])
stj_sts_df = stj_sts_df.dropna()

In [8]:
train, test = train_test_split(stj_sts_df, 
                            test_size=0.3, 
                            stratify=stj_sts_df.range,
                            shuffle=True,
                            random_state=42)
test, valid = train_test_split(test, 
                            test_size=0.3, 
                            stratify=test.range,
                            shuffle=True,
                            random_state=42)

In [9]:
stj_sts_df.loc[train.index.values,'SPLIT'] = 'TRAIN'
stj_sts_df.loc[test.index.values,'SPLIT'] = 'TEST'
stj_sts_df.loc[valid.index.values,'SPLIT'] = 'VALID'

In [10]:
print(stj_sts_df)
stj_sts_df.to_csv('../datasets/stj_sts.csv', index=False)

                                              sentence_A  \
0      ADMINISTRATIVO. RECURSO ORDINÁRIO EM MANDADO D...   
1      ADMINISTRATIVO. AGRAVO REGIMENTAL. RECURSO ORD...   
2      ADMINISTRATIVO. RECURSO ORDINÁRIO EM MANDADO D...   
3      ADMINISTRATIVO. AGRAVO REGIMENTAL. RECURSO ORD...   
4      ADMINISTRATIVO E PROCESSUAL CIVIL. SEGUNDOS EM...   
...                                                  ...   
54636  AGRAVO INTERNO NO RECURSO ESPECIAL. DIREITO CI...   
54637  AGRAVO INTERNO NO AGRAVO EM RECURSO ESPECIAL. ...   
54638           Data da Publicação/Fonte\nDJe 05/05/2011   
54639  CIVIL E PROCESSUAL CIVIL. AGRAVO REGIMENTAL. A...   
54640  Direito civil. Previdência privada. Benefícios...   

                                              sentence_B     score  range  \
0      ADMINISTRATIVO. RECURSO ORDINÁRIO EM MANDADO D...  4.676405      4   
1      ADMINISTRATIVO E PROCESSUAL CIVIL. SEGUNDOS EM...  4.540016      4   
2      ADMINISTRATIVO E PROCESSUAL CIVIL. SEGUND

# Análise de dados dos precedentes do TCU

In [11]:
tcu = pd.read_csv('../datasets/jurisprudencias_tcu_final.csv')

Gero pares de uma mesma Jurisprudência

In [12]:
tcu_sts_data = []
for group_name, jurisprudencia_group in tcu.groupby('ENUNCIADO'):
    pares = list(set(itertools.combinations(jurisprudencia_group.VOTO, 2)))
    noise = np.random.normal(mu, sigma, len(pares))
    scores = [4.5]*len(pares)+noise
    for i, par in enumerate(pares):
        tcu_sts_data.append([par[0], par[1], scores[i], 4])

precedentes_mesma_jurisprudencia = len(tcu_sts_data)
print('Total de pares de precedentes da mesma Jurisprudência gerados: ', precedentes_mesma_jurisprudencia)

Total de pares de precedentes da mesma Jurisprudência gerados:  1717


Gero pares de Áreas diferentes, então a similaridade é praticamente nula

In [13]:
pares_areas = list(set(itertools.combinations(tcu.AREA.unique().tolist(), 2)))
print('Total de pares de diferentes Áreas: ' , len(pares_areas))
total_por_par_areas = int(precedentes_mesma_jurisprudencia/len(pares_areas))
print('Total de pares a serem usados a partir de cada par de Matéria: ', total_por_par_areas)
for areas_par in list(set(itertools.combinations(tcu.AREA.unique().tolist(), 2))):
    #recupero os votos das duas diferentes áreas
    precedentes_areas = [tcu[tcu.AREA == area].VOTO.tolist() for area in areas_par]
    # gero um produto cartesiano entre os votos das diferentes áreas
    pares = list(itertools.product(*precedentes_areas))[:total_por_par_areas]
    noise = np.random.normal(mu, sigma, len(pares))
    scores = [0.5]*len(pares)+noise
    for i, par in enumerate(pares):
        tcu_sts_data.append([par[0], par[1], scores[i], 0])
print('Total de pares até o momento: ', len(tcu_sts_data))

Total de pares de diferentes Áreas:  6
Total de pares a serem usados a partir de cada par de Matéria:  286
Total de pares até o momento:  3433


In [14]:
pares_temas = []
for group_name, area_group in tcu.groupby('AREA'):
    for tema_name, tema_group in area_group.groupby('TEMA'):
        #Se pra um TEMA tenho mais que um subtema então a similaridade entre subtemas não é tão grande
        if len(tema_group.SUBTEMA.unique()) > 1:
            for subtemas_pares in list(set(itertools.combinations(tema_group.SUBTEMA.unique(), 2))):
                precedentes_subtemas = [tema_group[tema_group.SUBTEMA == subtema].VOTO.tolist() 
                                        for subtema in subtemas_pares]
                pares = list(itertools.product(*precedentes_subtemas))
                noise = np.random.normal(mu, sigma, len(pares))
                scores = [3]*len(pares)+noise
                for i, par in enumerate(pares):
                    pares_temas.append([par[0], par[1], scores[i], 3])


In [15]:
samples_pares_temas = random.sample(pares_temas, min(precedentes_mesma_jurisprudencia, len(pares_temas)))

tcu_sts_data = tcu_sts_data + samples_pares_temas
print('Total de pares até o momento', len(tcu_sts_data))

Total de pares até o momento 5030


In [16]:
tcu_sts_df = pd.DataFrame(tcu_sts_data, columns=['sentence_A','sentence_B','score','range'])
tcu_sts_df = tcu_sts_df.dropna()

In [17]:
train, test = train_test_split(tcu_sts_df, 
                            test_size=0.3, 
                            stratify=tcu_sts_df.range,
                            shuffle=True,
                            random_state=42)
test, valid = train_test_split(test, 
                            test_size=0.3, 
                            stratify=test.range,
                            shuffle=True,
                            random_state=42)

In [18]:
tcu_sts_df.loc[train.index.values,'SPLIT'] = 'TRAIN'
tcu_sts_df.loc[test.index.values,'SPLIT'] = 'TEST'
tcu_sts_df.loc[valid.index.values,'SPLIT'] = 'VALID'

In [19]:
print(tcu_sts_df)
tcu_sts_df.to_csv('../datasets/tcu_sts.csv', index=False)

                                             sentence_A  \
0      \n                      TRIBUNAL DE CONTAS DA...   
1      \n                      TRIBUNAL DE CONTAS DA...   
2      \n                      TRIBUNAL DE CONTAS DA...   
3      \n                      TRIBUNAL DE CONTAS DA...   
4      \n                      TRIBUNAL DE CONTAS DA...   
...                                                 ...   
5025  Versam estes autos acerca de relatório de audi...   
5026  Trata-se de atos de concessão de aposentadoria...   
5027  Trata-se de processo apartado de representação...   
5028   \n                      TRIBUNAL DE CONTAS DA...   
5029  Adoto como relatório, com os ajustes pertinent...   

                                             sentence_B     score  range  \
0      \n                      TRIBUNAL DE CONTAS DA...  4.574556      4   
1     Para verificar as assinaturas, acesse www.tcu....  4.541598      4   
2      \n                      TRIBUNAL DE CONTAS DA...  4.4667